## API 2.0

In [2]:
import requests
import pandas as pd
import random
from sklearn.feature_extraction.text import CountVectorizer
from time import sleep
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.utils import resample
from sklearn.model_selection import cross_val_score
import numpy as np
import collections
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
from sklearn.model_selection import GridSearchCV

### Poging 2

In [12]:

segments = requests.get("https://telraam-api.net/v0/segments/active")

In [25]:
print(segments.json().keys())

dict_keys(['type', 'features'])


In [32]:
y=segments.json()['features'][0]

In [33]:
print(y)

{'type': 'Feature', 'geometry': {'type': 'MultiLineString', 'coordinates': [[[3.961660422554976, 51.174096776333265], [3.9618278139267407, 51.174055412243966], [3.9619357558237867, 51.17403619759201], [3.9619511665926064, 51.17403538536309], [3.9620261713823197, 51.17403143392562], [3.962144636700207, 51.17403575354442], [3.962403181783646, 51.17404201713459], [3.9629303080170866, 51.174058082769285], [3.96314762036318, 51.17406542518445], [3.963324005387335, 51.17407617871948], [3.9637128126966785, 51.17410439527428], [3.9640980915348805, 51.174135375910765], [3.9650749768626277, 51.17422204721701], [3.965854294361505, 51.17428873851363], [3.966408800305373, 51.17433882309016], [3.9668630868045596, 51.17438536386801], [3.9672829231791766, 51.174435651861465], [3.9678883670933143, 51.17451400910348], [3.9687468690072403, 51.1746307384449], [3.9691372225799166, 51.174691382112044], [3.9693646055750222, 51.17473063919524], [3.9693935459184595, 51.17473563591953], [3.969486001031014, 51.1

In [14]:

x=segments.json()['features'][0]['properties']

In [15]:
x.keys()

dict_keys(['id', 'speed', 'oneway', 'road_type', 'road_speed', 'pedestrian', 'bike', 'car', 'lorry', 'last_data_package', 'pedestrian_avg', 'bike_avg', 'car_avg', 'lorry_avg', 'typical_data', 'car_speed_histogram', 'car_speed_buckets'])

In [16]:
print(x)

{'id': 377443, 'speed': 50, 'oneway': False, 'road_type': '', 'road_speed': '', 'pedestrian': None, 'bike': None, 'car': None, 'lorry': None, 'last_data_package': None, 'pedestrian_avg': 0, 'bike_avg': 0, 'car_avg': 0, 'lorry_avg': 0, 'typical_data': None, 'car_speed_histogram': None, 'car_speed_buckets': None}


In [17]:
pd.DataFrame([x])

,id,speed,oneway,road_type,road_speed,pedestrian,bike,car,lorry,last_data_package,pedestrian_avg,bike_avg,car_avg,lorry_avg,typical_data,car_speed_histogram,car_speed_buckets
0,377443,50,False,,,None,None,None,None,None,0,0,0,0,None,None,None


### now create a function to do this

In [18]:
def dataline(nr):
    x=segments.json()['features'][nr]['properties']
    lijntje=pd.DataFrame([x])
    return(lijntje)
    

In [21]:
dataline(5)

,id,speed,oneway,road_type,road_speed,pedestrian,bike,car,lorry,last_data_package,pedestrian_avg,bike_avg,car_avg,lorry_avg,typical_data,car_speed_histogram,car_speed_buckets
0,2000026769,50,False,,,72,226,449,44,None,1,11,23,0,"[{'hour': '05:00:00', 'pedestrian': None, 'bik...","[157, 182, 88, 10, 7, 7]","[0, 1, 2, 3, 4, 7]"


## create a dataframe

In [38]:
df = pd.DataFrame()
for obs in range(0,100):
    l=dataline(obs)
    df=pd.concat([df, l])
print(df)

            id  speed  oneway road_type road_speed pedestrian  bike   car  \
0       377443     50   False                            None  None  None   
0   9000000005     50   False                             100   381  2168   
0   3000136655     50   False                            None  None  None   
0   3000129228     50   False                            None  None  None   
0       156203     50   False                            None  None  None   
..         ...    ...     ...       ...        ...        ...   ...   ...   
0        84783     50   False                              40    99    70   
0       279386     50   False                              29   189  3404   
0       350400     50   False                              18   441  2181   
0       350416     50   False                              49   347  1793   
0       379611     50   False                              18   198  1483   

   lorry         last_data_package  pedestrian_avg  bike_avg  car_avg  \
0 

In [39]:
df

,id,speed,oneway,road_type,road_speed,pedestrian,bike,car,lorry,last_data_package,pedestrian_avg,bike_avg,car_avg,lorry_avg,typical_data,car_speed_histogram,car_speed_buckets
0,377443,50,False,,,None,None,None,None,None,0,0,0,0,None,None,None
0,9000000005,50,False,,,100,381,2168,3336,None,2,19,98,100,"[{'hour': '05:00:00', 'pedestrian': None, 'bik...","[7, 110, 193, 337, 478, 537, 358, 152]","[0, 1, 2, 3, 4, 5, 6, 7]"
0,3000136655,50,False,,,None,None,None,None,None,0,0,0,0,None,None,None
0,3000129228,50,False,,,None,None,None,None,None,0,0,0,0,None,None,None
0,156203,50,False,,,None,None,None,None,None,0,0,0,0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,84783,50,False,,,40,99,70,0,2019-06-09T12:02:45.044Z,9,36,16,0,"[{'hour': '05:00:00', 'pedestrian': 0, 'bike':...",None,None
0,279386,50,False,,,29,189,3404,57,2019-06-10T08:53:33.950Z,5,29,282,5,"[{'hour': '05:00:00', 'pedestrian': 0, 'bike':...",None,None
0,350400,50,False,,,18,441,2181,87,2019-06-11T23:52:25.118Z,0,7,60,0,"[{'hour': '05:00:00', 'pedestrian': 0, 'bike':...",None,None
0,350416,50,False,,,49,347,1793,199,2019-06-12T23:58:18.607Z,0,42,84,0,"[{'hour': '05:00:00', 'pedestrian': 0, 'bike':...",None,None
